In [84]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
import numpy as np

In [85]:
# nltk.download('punkt')
# nltk.download('stopwords')

Read the Data

In [86]:
twitter_2013dev_a = pd.read_csv('dataset/twitter-2013dev-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2013test_a = pd.read_csv('dataset/twitter-2013test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2013train_a = pd.read_csv('dataset/twitter-2013train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2014sarcasm_a = pd.read_csv('dataset/twitter-2014sarcasm-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2014test_a = pd.read_csv('dataset/twitter-2014test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2015test_a = pd.read_csv('dataset/twitter-2015test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2015train_a = pd.read_csv('dataset/twitter-2015train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016dev_a = pd.read_csv('dataset/twitter-2016dev-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016devtest_a = pd.read_csv('dataset/twitter-2016devtest-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016test_a = pd.read_csv('dataset/twitter-2016test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence", "dates"])
twitter_2016train_a = pd.read_csv('dataset/twitter-2016train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])

data_df = pd.DataFrame()
# data_df = pd.concat([twitter_2013dev_a, twitter_2013test_a, twitter_2013train_a, twitter_2014sarcasm_a, twitter_2014test_a,
#                      twitter_2015test_a, twitter_2015train_a, twitter_2016dev_a, twitter_2016devtest_a,
#                      twitter_2016test_a[["id", "sentiment", "sentence"]], twitter_2016train_a])

data_df = pd.concat([twitter_2013dev_a, twitter_2013test_a])
data_df.head()

,id,sentiment,sentence
0,260097528899452929,neutral,Won the match #getin . Plus\u002c tomorrow is ...
1,263791921753882624,neutral,Some areas of New England could see the first ...
2,264194578381410304,negative,@francesco_con40 2nd worst QB. DEFINITELY Tony...
3,264041328420204544,neutral,#Thailand Washington - US President Barack Oba...
4,263816256640126976,neutral,Did y\u2019all hear what Tony Romo dressed up ...


Check for null values on the data

In [87]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5201 entries, 0 to 3546
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5201 non-null   int64 
 1   sentiment  5201 non-null   object
 2   sentence   5201 non-null   object
dtypes: int64(1), object(2)
memory usage: 162.5+ KB


In [88]:
data_df.isnull().values.any()

False

Drop the ID column and reset the indexes

In [89]:
data_df.drop("id", axis=1, inplace=True)
data_df.reset_index(inplace=True, drop=True)
data_df.head()

,sentiment,sentence
0,neutral,Won the match #getin . Plus\u002c tomorrow is ...
1,neutral,Some areas of New England could see the first ...
2,negative,@francesco_con40 2nd worst QB. DEFINITELY Tony...
3,neutral,#Thailand Washington - US President Barack Oba...
4,neutral,Did y\u2019all hear what Tony Romo dressed up ...


In [90]:
def remove_irr_char(sentence):
    """
        This function removes irrelevant characters such as punctuations and etc.
        Accepts a string sentence
        Returns a string cleaned sentence
    """
    word_list = []
    tokenized_sent = nltk.word_tokenize(sentence)
    for i in tokenized_sent:
        word_list.append(''.join(j for j in i if i.isalnum()))

    word_list = list(filter(None,  word_list))

    return ' '.join(word_list)

In [91]:
data_df["cleaned_sentence"] = data_df.apply(lambda x: remove_irr_char(x[1]), axis=1)
data_df.head()

,sentiment,sentence,cleaned_sentence
0,neutral,Won the match #getin . Plus\u002c tomorrow is ...,Won the match getin tomorrow is a very busy wi...
1,neutral,Some areas of New England could see the first ...,Some areas of New England could see the first ...
2,negative,@francesco_con40 2nd worst QB. DEFINITELY Tony...,2nd worst QB DEFINITELY Tony Romo The man who ...
3,neutral,#Thailand Washington - US President Barack Oba...,Thailand Washington US President Barack Obama ...
4,neutral,Did y\u2019all hear what Tony Romo dressed up ...,Did hear what Tony Romo dressed up as for Hall...


In [92]:
def remove_stop_words(sentence):
    """
        This function removes stop words from the cleaned sentence
        Accepts a string
        Returns a string without the stop words
    """
    tokenized = nltk.word_tokenize(sentence)
    stop_words = set({"a", "about", "an", "are", "as", "at", "be", "by", "com", "de", "en", "for", "from", "how", "i", "in",
                      "is", "it", "la", "of", "on", "or", "that", "this", "to", "was", "what", "when", "where", "who", "will",
                      "with", "und", "the", "www"})

    filtered_list = [
        word for word in tokenized if word.casefold() not in stop_words
    ]

    return ' '.join(filtered_list)

In [93]:
data_df["stopwords_remove_sentence"] = data_df.apply(lambda x: remove_stop_words(x[2]), axis=1)
data_df.head()

,sentiment,sentence,cleaned_sentence,stopwords_remove_sentence
0,neutral,Won the match #getin . Plus\u002c tomorrow is ...,Won the match getin tomorrow is a very busy wi...,Won match getin tomorrow very busy Awareness a...
1,neutral,Some areas of New England could see the first ...,Some areas of New England could see the first ...,Some areas New England could see first flakes ...
2,negative,@francesco_con40 2nd worst QB. DEFINITELY Tony...,2nd worst QB DEFINITELY Tony Romo The man who ...,2nd worst QB DEFINITELY Tony Romo man likes sh...
3,neutral,#Thailand Washington - US President Barack Oba...,Thailand Washington US President Barack Obama ...,Thailand Washington US President Barack Obama ...
4,neutral,Did y\u2019all hear what Tony Romo dressed up ...,Did hear what Tony Romo dressed up as for Hall...,Did hear Tony Romo dressed up Halloween Giants...


In [94]:
stemmer = PorterStemmer()

def stem_tokens(sentence):
    """
        This function stems the sentence to convert each tokens to their root base
        Accepts a string
        Returns a string sentence in their root base
    """
    word_list = []
    tokenized_sent = nltk.word_tokenize(sentence)
    for i in tokenized_sent:
        stem_word = stemmer.stem(i)
        word_list.append(stem_word)

    return ' '.join(word_list)

In [95]:
data_df["stemmed_sentence"] = data_df.apply(lambda x: stem_tokens(x[3]), axis=1)
data_df.head()

,sentiment,sentence,cleaned_sentence,stopwords_remove_sentence,stemmed_sentence
0,neutral,Won the match #getin . Plus\u002c tomorrow is ...,Won the match getin tomorrow is a very busy wi...,Won match getin tomorrow very busy Awareness a...,won match getin tomorrow veri busi awar and de...
1,neutral,Some areas of New England could see the first ...,Some areas of New England could see the first ...,Some areas New England could see first flakes ...,some area new england could see first flake se...
2,negative,@francesco_con40 2nd worst QB. DEFINITELY Tony...,2nd worst QB DEFINITELY Tony Romo The man who ...,2nd worst QB DEFINITELY Tony Romo man likes sh...,2nd worst qb definit toni romo man like share ...
3,neutral,#Thailand Washington - US President Barack Oba...,Thailand Washington US President Barack Obama ...,Thailand Washington US President Barack Obama ...,thailand washington us presid barack obama vow...
4,neutral,Did y\u2019all hear what Tony Romo dressed up ...,Did hear what Tony Romo dressed up as for Hall...,Did hear Tony Romo dressed up Halloween Giants...,did hear toni romo dress up halloween giant qu...


Let us check for the value distribution of the sentiment in our data

In [96]:
data_df["sentiment"].value_counts()

sentiment
neutral     2252
positive    2050
negative     899
Name: count, dtype: int64

Let us convert the sentiment column to numerical (1 = Positive, 0 = Neutral, -1 = Negative)

In [97]:
data_df["sentiment"] = np.where((data_df["sentiment"] == "positive"), +1, data_df["sentiment"])
data_df["sentiment"] = np.where((data_df["sentiment"] == "negative"), -1, data_df["sentiment"])
data_df["sentiment"] = np.where((data_df["sentiment"] == "neutral"), 0, data_df["sentiment"])

In [98]:
data_df["sentiment"].value_counts()

sentiment
0     2252
1     2050
-1     899
Name: count, dtype: int64

Final cleaned data

In [99]:
data_df.head()

,sentiment,sentence,cleaned_sentence,stopwords_remove_sentence,stemmed_sentence
0,0,Won the match #getin . Plus\u002c tomorrow is ...,Won the match getin tomorrow is a very busy wi...,Won match getin tomorrow very busy Awareness a...,won match getin tomorrow veri busi awar and de...
1,0,Some areas of New England could see the first ...,Some areas of New England could see the first ...,Some areas New England could see first flakes ...,some area new england could see first flake se...
2,-1,@francesco_con40 2nd worst QB. DEFINITELY Tony...,2nd worst QB DEFINITELY Tony Romo The man who ...,2nd worst QB DEFINITELY Tony Romo man likes sh...,2nd worst qb definit toni romo man like share ...
3,0,#Thailand Washington - US President Barack Oba...,Thailand Washington US President Barack Obama ...,Thailand Washington US President Barack Obama ...,thailand washington us presid barack obama vow...
4,0,Did y\u2019all hear what Tony Romo dressed up ...,Did hear what Tony Romo dressed up as for Hall...,Did hear Tony Romo dressed up Halloween Giants...,did hear toni romo dress up halloween giant qu...


### Logistic Regression Model without feature engineering using  CountVectorizer

In [100]:
data_mod = data_df[["sentiment", "stemmed_sentence"]]
data_mod.head()

,sentiment,stemmed_sentence
0,0,won match getin tomorrow veri busi awar and de...
1,0,some area new england could see first flake se...
2,-1,2nd worst qb definit toni romo man like share ...
3,0,thailand washington us presid barack obama vow...
4,0,did hear toni romo dress up halloween giant qu...


In [101]:
from sklearn.feature_extraction.text import CountVectorizer

In [102]:
vectorizer_count = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    min_df = 3
)

features = vectorizer_count.fit_transform(
    data_mod["stemmed_sentence"]
)

features_nd = features.toarray()

In [103]:
data_labels = []
for i in data_mod["sentiment"]:
    data_labels.append(i)

In [104]:
from sklearn.model_selection import train_test_split

X_train_cv, X_test_cv, y_train_cv, y_test_cv  = train_test_split(
        features_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

In [105]:
from sklearn.linear_model import LogisticRegression
log_model_1 =  LogisticRegression()

In [106]:
log_model_1 = log_model_1.fit(X_train_cv, y_train_cv)

c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [107]:
y_pred_cv = log_model_1.predict(X_test_cv)

In [108]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test_cv, y_pred_cv))

0.6195965417867435


### Logistic Regression Model without feature engineering using TFIDF Vectorizer

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [110]:
vectorizer_tfidf = TfidfVectorizer(min_df=3)

features2 = vectorizer_tfidf.fit_transform(data_mod["stemmed_sentence"])

features_nd2 = features2.toarray()


In [111]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf  = train_test_split(
        features_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

In [112]:
log_model_2 =  LogisticRegression()

In [113]:
log_model_2 = log_model_2.fit(X_train_tfidf, y_train_tfidf)

c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [114]:
y_pred_tfidf = log_model_2.predict(X_test_tfidf)
print(accuracy_score(y_test_tfidf, y_pred_tfidf))

0.6195965417867435


### Naive Bayes Model without Feature Engineering

In [115]:
from sklearn.naive_bayes import MultinomialNB

#### Using CountVectorizer

In [116]:
multinomialNB = MultinomialNB(force_alpha=True)

multinomialNB.fit(X_train_cv, y_train_cv)

MultinomialNB(force_alpha=True)

In [117]:
y_pred_nb_cv = multinomialNB.predict(X_test_cv)
print(accuracy_score(y_test_cv, y_pred_nb_cv))

0.6042267050912584


Using TF-IDF Vectorizer

In [118]:
multinomialNB2 = MultinomialNB(force_alpha=True)

multinomialNB2.fit(X_train_tfidf, y_train_tfidf)

y_pred_nb_tfidf = multinomialNB2.predict(X_test_tfidf)
print(accuracy_score(y_test_tfidf, y_pred_nb_tfidf))

0.6042267050912584


### Logistic Regression Models with Oversampling

In [119]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [120]:
oversample = RandomOverSampler(sampling_strategy='minority')

X_over, y_over = oversample.fit_resample(features_nd, data_labels)
print(Counter(y_over))

Counter({0: 2252, -1: 2252, 1: 2050})


In [121]:
X_train, X_test, y_train, y_test  = train_test_split(
        X_over, 
        y_over,
        train_size=0.80, 
        random_state=10)

In [122]:
log_model_3 = LogisticRegression()
log_model_3 = log_model_3.fit(X_train, y_train)

c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [123]:
y_pred = log_model_3.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7391304347826086
              precision    recall  f1-score   support

          -1       0.78      0.90      0.84       462
           0       0.68      0.70      0.69       436
           1       0.75      0.60      0.66       413

    accuracy                           0.74      1311
   macro avg       0.74      0.73      0.73      1311
weighted avg       0.74      0.74      0.73      1311



Lets apply GridSearch to get the best hyperparameter

In [124]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'C': np.logspace(-5, 8, 15),
    'max_iter': [1,2,3,4,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]
}

grid_search = LogisticRegression()

grid_search = GridSearchCV(
    grid_search,
    parameters,
    cv = 5,
    scoring = 'accuracy',
    n_jobs=-1
)

grid_result = grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_result.best_params_)

Best parameters:  {'C': 3.727593720314938, 'max_iter': 40}


c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [125]:
log_model_4 = LogisticRegression(C = 3.727593720314938, max_iter=40)
log_model_4.fit(X_train, y_train)

c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=3.727593720314938, max_iter=40)

In [126]:
y_pred = log_model_4.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7429443173150267
              precision    recall  f1-score   support

          -1       0.79      0.92      0.85       462
           0       0.70      0.67      0.68       436
           1       0.73      0.61      0.67       413

    accuracy                           0.74      1311
   macro avg       0.74      0.74      0.73      1311
weighted avg       0.74      0.74      0.74      1311



### Models using stopwords removed sentence

In [157]:
data_mod = data_df[["sentiment", "stopwords_remove_sentence"]]
data_mod.head()

,sentiment,stopwords_remove_sentence
0,0,Won match getin tomorrow very busy Awareness a...
1,0,Some areas New England could see first flakes ...
2,-1,2nd worst QB DEFINITELY Tony Romo man likes sh...
3,0,Thailand Washington US President Barack Obama ...
4,0,Did hear Tony Romo dressed up Halloween Giants...


In [158]:
vectorizer_count = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    min_df = 3
)

features_sr = vectorizer_count.fit_transform(
    data_mod["stopwords_remove_sentence"]
)

features_sr_nd = features.toarray()

In [159]:
data_labels = []
for i in data_mod["sentiment"]:
    data_labels.append(i)

In [160]:
oversample_sr = RandomOverSampler(sampling_strategy='minority')

X_over_sr, y_over_sr = oversample.fit_resample(features_sr_nd, data_labels)
print(Counter(y_over_sr))

Counter({0: 2252, -1: 2252, 1: 2050})


In [161]:
X_train_sr, X_test_sr, y_train_sr, y_test_sr  = train_test_split(
        X_over_sr, 
        y_over_sr,
        train_size=0.80, 
        random_state=10)

In [162]:
log_model_5 = LogisticRegression(C = 3.727593720314938, max_iter=40)
log_model_5 = log_model_5.fit(X_train_sr, y_train_sr)

c:\Training\Project_NLP\project_env2\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [163]:
y_pred = log_model_5.predict(X_test_sr)
print(accuracy_score(y_test_sr, y_pred))
print(classification_report(y_test_sr, y_pred))

0.7376048817696415
              precision    recall  f1-score   support

          -1       0.79      0.91      0.85       462
           0       0.68      0.68      0.68       436
           1       0.72      0.61      0.66       413

    accuracy                           0.74      1311
   macro avg       0.73      0.73      0.73      1311
weighted avg       0.73      0.74      0.73      1311



In [142]:
import joblib

In [164]:
model_filename = "log_reg_oversampling_ht_sr.joblib"
# vectorizer_filename = "tfidf_vectorizer.joblib"
joblib.dump(log_model_4, "nlp_project/models/" + model_filename)
# joblib.dump(vectorizer_tfidf, "nlp_project/models/" + vectorizer_filename)

['nlp_project/models/log_reg_oversampling_ht_sr.joblib']

In [154]:
# print(log_model_3.predict(vectorizer_tfidf.transform(nltk.word_tokenize("I'm not happy")).toarray()))
# print(log_model_3.predict(vectorizer_tfidf.transform("I am happy")))

test_pred = log_model_4.predict(vectorizer_count.transform(["I am very"]).toarray())
print(test_pred[0])
print(y_pred[0])

1
-1


In [156]:
type(vectorizer_count.transform(["Congratulations on your win"]).toarray())

numpy.ndarray

In [128]:
# print(X_test[0])
# print(vectorizer_tfidf.transform(nltk.word_tokenize("I'm not happy")).toarray())

In [129]:
# print(data_df["stemmed_sentence"].where(data_df["sentiment"] > 0).dropna())
# print(data_df)